In [ ]:
from autogen import UserProxyAgent, ConversableAgent

local_llm_config = {
    "config_list": [
        {
            "model": "NotRequired",  # Loaded with LiteLLM command
            "api_key": "NotRequired",  # Not needed
            "base_url": "http://0.0.0.0:34796",  # Your LiteLLM URL
        }
    ],
    "cache_seed": None,  # Turns off caching, useful for testing different models
}

# Create the agent that uses the LLM.
assistant = ConversableAgent("agent", llm_config=local_llm_config)

# Create the agent that represents the user in the conversation.
user_proxy = UserProxyAgent("user", code_execution_config=False)

# Let the assistant start the conversation.  It will end when the user types exit.
assistant.initiate_chat(user_proxy, message="How can I help you today?")

In [ ]:
from typing import Annotated, Literal

Operator = Literal["+", "-", "*", "/"]


def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return int(a / b)
    else:
        raise ValueError("Invalid operator")

In [ ]:
from autogen import register_function

# Register the calculator function to the two agents.
register_function(
    calculator,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="calculator",  # By default, the function name is used as the tool name.
    description="A simple calculator",  # A description of the tool.
)

In [ ]:
from litellm import completion

response = completion(
    model="ollama/phi3:latest",
    messages=[{"content": "respond in 20 words. who are you?", "role": "user"}],
    api_base="http://localhost:11434",
    stream=True,
)
print(response)
for chunk in response:
    print(chunk["choices"][0]["delta"])

In [ ]:
from langchain_community.chat_models import ChatLiteLLM
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio",
    model="NousResearch/Hermes-2-Pro-Mistral-7B-GGUF",
)

In [ ]:
messages = [
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    )
]
chat.invoke('hi')

In [ ]:
from langchain import hub
from langchain.agents import (
    AgentExecutor,
    create_openai_tools_agent,
    create_json_chat_agent,
)
from langchain_core.tools import tool

@tool
def multiply(a: float, b: float) -> int:
    """Multiply two numbers."""
    return a * b

tools = [multiply]
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react-chat-json")
agent = create_json_chat_agent(chat, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "how much is 1323 times 5827?"})

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

query = "What is 3 * 12? Also, what is 11 + 49?"

tools = [add, multiply]
llm_with_tools = chat.bind_tools(tools)

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

messages